# Recommendation System

## Предобработка данных

Объединим общие данные о фильмах [tmdb_5000_movies](Recommendation-system\datasets\tmdb_5000_movies.csv) и каст фильмов
[tmdb_5000_credits](Recommendation-system\datasets\tmdb_5000_credits.csv). Оставим в датасете только фильмы, которые вышли в "релиз".

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('../datasets/tmdb_5000_movies.csv')
credits = pd.read_csv('../datasets/tmdb_5000_credits.csv')

In [3]:
new_movies = pd.merge(movies, credits, how='left', left_on='id', right_on='movie_id')
new_movies = new_movies[new_movies.status == 'Released'].reset_index()

In [4]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


## Алгоритмы для построения рекомендательных систем

### На основе оценок пользователей

Используем текущую рейтинговую систему IMDB (weighted rating (WR)):
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов

Выведем ТОП-5 фильмов

In [5]:
def weighted_rating(df):
    v =  df.vote_count
    m = df.vote_count.quantile(.95)
    r = df.vote_average
    c = df.vote_average.mean()
    df['simple_score'] = v / (v + m) * r + m / (v + m) * c
    return df

In [6]:
new_movies = weighted_rating(new_movies)
new_movies.sort_values(by='simple_score', ascending=False)[['original_title', 'simple_score']][:5]

,original_title,simple_score
1881,The Shawshank Redemption,7.849025
65,The Dark Knight,7.773990
662,Fight Club,7.761012
96,Inception,7.736496
3231,Pulp Fiction,7.714726


### На основе содержания

Проведем предобработку текста:
* Заменим NaN в описании фильма на пустой символ `''`
* Удалим все английские стоп слова
* Расcчитаем матрица Tf-Idf
* Рассчитаем матрицу косинусного расстояния

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
new_movies['overview'] = new_movies['overview'].fillna('')
tfidf = TfidfVectorizer(stop_words='english')
movies_tfidf = tfidf.fit_transform(new_movies['overview'])


In [9]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(movies_tfidf)

Напишем функцию, рекомендующую K фильмов, чьи описания похожи на выбранный фильм. 
Проверим ее, выведя топ-5 фильмов для фильма `Saving Private Ryan`

In [10]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k):
    """
    1. Находим индекс выбранного фильма
    2. Находим в матрице расстояний столбец для фильма
    3. Сортируем этот столбец по убыванию(т.к. чем ближе значение косинусного сходства к 1,
    тем векторы более похожи друг на друга(т.е. описания фильмов более схожи между собой)),
    выбираем нужное нам количество записей и находим их индексы
    4. По найденным индексам возвращаем список фильмов
    """
    movie_index = movies_dataset.index[movies_dataset['original_title'] == title]
    movie_cos_sim = pd.DataFrame(cosine_sim[:, movie_index])
    movie_cos_sim.columns = ['cosine_sim']
    new_movies['cosine_sim'] = movie_cos_sim
    new_movies.sort_values(['cosine_sim'], ascending=False)[['original_title', 'cosine_sim']]
    movie_sim_index = movie_cos_sim.sort_values(by='cosine_sim', ascending=False).head(top_k+1).tail(top_k).index
    return movies_dataset.iloc[movie_sim_index]['original_title']

In [11]:
get_recommendations(movies_dataset=new_movies, title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5)

787        The Great Raid
586     The Monuments Men
290     The Expendables 2
4167            Abandoned
3515            The Train
Name: original_title, dtype: object

### На основе сходства между пользователями

Для реализации коллаборативной фильтрации потребуются оценки пользователей [ratings](../datasets/ratings.csv).

Используем библиотеку `surprise` для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD).

In [12]:
ratings = pd.read_csv('../datasets/ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [13]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader=reader)
# Run 5-fold cross-validation and print results.
res = cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f'Среднее значение RMSE = {res["test_rmse"].mean().round(4)} \n'
      f'Среднее значение MAE = {res["test_mae"].mean().round(4)}')


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8928  0.8919  0.9083  0.8962  0.8919  0.8962  0.0062  
MAE (testset)     0.6878  0.6847  0.6966  0.6891  0.6890  0.6894  0.0039  
Fit time          0.73    0.75    0.77    0.72    0.73    0.74    0.02    
Test time         0.16    0.08    0.07    0.12    0.08    0.10    0.03    
Среднее значение RMSE = 0.8962 
Среднее значение MAE = 0.6894
